In [66]:
#import externally developed packages
import os
import sys
import vcfpy
import re
import pandas as pd

## Import a .vcf file and convert it into a dataframe with a unique identifier for each SNP

In [65]:
##import vcf

#specify path of vcf
vcf_path = '/Users/tristanpwdennis/Software/seqtoolbox/analysis-scripts/vcf/vcf/test.vcf'

#import vcf as reader object
reader = vcfpy.Reader.from_path(vcf_path)

# Build header
header = pd.DataFrame.transpose(pd.DataFrame(['#CHROM', 'POS', 'REF', 'ALT'] + reader.header.samples.names))

# Design matrix in list format
X_list = []  

#convert vcf convert to dataframe - extract GT field, chrom, pos ref and alt data
for record in reader:
    if not record.is_snv():
        continue
    line = [record.CHROM, record.POS, record.REF]
    line += [alt.value for alt in record.ALT]
    line += [call.data.get('GT') or './.' for call in record.calls]
    X_list.append(line)
    df = pd.DataFrame(X_list)

#stick header onto the genotype frame
df=header.append(df) 

#make added row into the df header
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

#make unique snp id
df["id"] = df["#CHROM"].map(str) + df["POS"].map(str) + df["REF"].map(str) + df["ALT"].map(str)

##have a wee look at the finished dataframe
df.head()

,#CHROM,POS,REF,ALT,ind0,ind1,ind2,ind3,ind4,ind5,...,ind42,ind43,ind44,ind45,ind46,ind47,ind48,ind49,ind50,id
0,OVOC_OO_000327,1374,C,T,0/0,0/1,0/0,0/0,0/1,0/1,...,0/0,0/0,0/0,0/0,0/0,0/1,0/1,0/1,0/0,OVOC_OO_0003271374CT
1,OVOC_OO_000327,1375,G,A,0/0,0/0,0/1,0/0,0/1,0/1,...,0/0,0/0,0/0,0/1,0/0,0/0,0/1,0/0,0/0,OVOC_OO_0003271375GA
2,OVOC_OO_000327,1571,A,C,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,OVOC_OO_0003271571AC
3,OVOC_OO_000327,1612,T,C,0/0,0/0,0/0,0/1,0/0,0/0,...,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0,OVOC_OO_0003271612TC
4,OVOC_OO_000327,1629,G,T,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/0,OVOC_OO_0003271629GT


In [60]:
##vcf to colony input

#specify mapping (from genotype values to codes)
di = {
    "./.": 0,
    "0/0": 1,
    "0/1": 2,
    "1/0": 3,
    "1/1": 4,
}  

#make genotype id

df.replace({"col1": di})


In [50]:
vcf

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,#CHROM,POS,REF,ALT,ind0,ind1,ind2,ind3,ind4,ind5,...,ind41,ind42,ind43,ind44,ind45,ind46,ind47,ind48,ind49,ind50
0,OVOC_OO_000327,1374,C,T,0/0,0/1,0/0,0/0,0/1,0/1,...,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/1,0/1,0/0
1,OVOC_OO_000327,1375,G,A,0/0,0/0,0/1,0/0,0/1,0/1,...,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/1,0/0,0/0
2,OVOC_OO_000327,1571,A,C,0/0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0
3,OVOC_OO_000327,1612,T,C,0/0,0/0,0/0,0/1,0/0,0/0,...,0/0,0/0,0/0,0/1,0/0,0/0,0/0,0/0,0/1,0/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,OVOC_OO_000024,11432,T,A,0/1,0/1,0/0,1/1,0/0,1/1,...,0/1,0/1,0/0,1/1,0/1,1/1,0/0,0/0,1/1,0/1
83,OVOC_OO_000024,11443,T,C,0/1,0/1,0/1,0/1,0/0,1/1,...,0/1,0/1,0/0,1/1,1/1,1/1,0/0,0/0,1/1,0/1
84,OVOC_OO_000024,11561,C,T,0/0,0/1,0/1,0/1,1/1,0/0,...,0/1,1/1,0/1,0/1,0/0,0/0,0/1,1/1,0/0,0/1
85,OVOC_OO_000024,11823,A,T,0/1,0/1,1/1,0/1,1/1,0/1,...,0/1,0/0,0/1,0/1,0/0,0/0,1/1,0/0,0/0,0/1
